# EduSync Component Specification
## Teacher Dashboard Component

**Document Version:** 1.0
**Date:** February 20, 2026
**Author:** Kyle Sherman
**Component Status:** Design Phase

---

## Table of Contents

1. [Top-Down Component Architecture](#top-down-component-architecture)
2. [Teacher Dashboard Overview](#teacher-dashboard-overview)
3. [Component Specification](#component-specification)
4. [Logic Model](#logic-model)
5. [Source Code Implementation](#source-code-implementation)
6. [Testing Strategy](#testing-strategy)

---

## 1. Top-Down Component Architecture

### 1.1 System Hierarchy

```
EduSync Platform
│
├── Presentation Layer
│   ├── Teacher Dashboard ◄── FOCUS COMPONENT
│   │   ├── Assignment Timeline View
│   │   ├── Platform Filter Panel
│   │   ├── Date Range Selector
│   │   ├── Quick Actions Menu
│   │   └── Sync Status Indicator
│   │
│   ├── Student Portal
│   │   ├── Student Progress View
│   │   ├── Missing Assignments List
│   │   └── Grade Summary Cards
│   │
│   └── Settings Console
│       ├── Platform Connection Manager
│       ├── Course Mapping Interface
│       └── User Profile Settings
│
├── Application Layer
│   ├── Assignment Aggregation Service
│   ├── Student Analytics Service
│   ├── Sync Orchestration Service
│   └── Authentication Service
│
├── Integration Layer
│   ├── Platform Adapter Registry
│   ├── Google Classroom Adapter
│   ├── PowerSchool Adapter
│   └── CodeHS Adapter
│
└── Data Layer
    ├── PostgreSQL (Primary Storage)
    ├── Redis (Cache)
    └── Celery (Task Queue)
```

### 1.2 Component Relationships

**Teacher Dashboard Dependencies:**
- **Upstream (Data Sources):**
  - Assignment Aggregation Service (provides unified assignment data)
  - Platform Adapter Registry (provides connection status)
  - Sync Orchestration Service (provides sync timestamps)
  - Authentication Service (provides user context)

- **Downstream (Data Consumers):**
  - User interactions trigger API calls to Application Layer
  - Filter selections update query parameters
  - Navigation events route to other presentation components

- **Peer Components:**
  - Student Portal (shared data models)
  - Settings Console (configuration changes affect dashboard state)

### 1.3 Component Boundaries

**In Scope:**
- Display aggregated assignments from multiple platforms
- Filter assignments by date, platform, course, type
- Visual indicators for due dates, platforms, submission status
- Click-through links to native platform interfaces
- Real-time sync status display
- Export to calendar functionality

**Out of Scope:**
- Assignment creation (handled by native platforms)
- Grade entry (handled by native platforms)
- Student-facing views (separate component)
- Administrative reports (separate component)

---

## 2. Teacher Dashboard Overview

### 2.1 Purpose Statement

The Teacher Dashboard component serves as the primary interface for educators to view and manage assignments across multiple ed-tech platforms in a unified timeline view. It eliminates context-switching overhead by aggregating data from Google Classroom, PowerSchool, CodeHS, and other connected platforms into a single, filterable interface.

### 2.2 User Stories Addressed

**Primary User Story:**
> As a CS teacher managing multiple platforms (AP Students, Google Classroom, CodeHS, PowerSchool), I want to view all upcoming assignments and deadlines from all platforms in a single unified dashboard, so that I can quickly assess workload distribution and identify scheduling conflicts without switching between platforms.

**Secondary User Stories:**
- Filter assignments by specific date ranges to focus on grading periods
- Identify platform-specific assignment clusters to balance workload
- Access original platform assignments for editing with single click
- Export assignment calendar to Google Calendar for offline reference

### 2.3 Key Features

1. **Unified Timeline View:** Chronological display of all assignments regardless of source platform
2. **Multi-Dimensional Filtering:** Date range, platform, course, assignment type filters
3. **Visual Platform Indicators:** Color-coded badges and icons for quick platform identification
4. **Due Date Proximity Warnings:** Visual cues for assignments due within 48 hours
5. **Connection Status Monitoring:** Real-time display of platform connection health
6. **Quick Actions:** Export, refresh, and navigate to platform actions
7. **Responsive Design:** Tablet and desktop optimized layout

### 2.4 Non-Functional Requirements

- **Performance:** Dashboard loads within 2 seconds with cached data
- **Responsiveness:** Supports viewport widths from 768px to 1920px
- **Accessibility:** WCAG 2.1 AA compliant (keyboard navigation, screen reader support)
- **Browser Support:** Chrome 90+, Firefox 88+, Safari 14+, Edge 90+
- **Data Freshness:** Displays last sync timestamp, maximum staleness 24 hours

---

## 3. Component Specification

### 3.1 Component Docstring

```python
"""
Teacher Dashboard Component

A React-based presentation component that aggregates and displays assignments
from multiple ed-tech platforms in a unified, filterable timeline interface.

The Teacher Dashboard is the primary landing page for educators using EduSync,
providing at-a-glance visibility into assignment distribution, due dates, and
platform connection status across Google Classroom, PowerSchool, CodeHS, and
other integrated platforms.

Responsibilities:
    - Fetch and display aggregated assignment data from API
    - Provide interactive filtering controls (date, platform, course, type)
    - Render assignment cards with platform-specific styling
    - Display sync status and last update timestamps
    - Handle user interactions (clicks, filter changes, exports)
    - Manage loading states and error conditions
    - Navigate to external platform URLs on assignment click

Dependencies:
    - React 18.x for component lifecycle and hooks
    - React Query for data fetching and caching
    - TailwindCSS for styling
    - date-fns for date manipulation
    - lucide-react for iconography
    - EduSync API for backend integration

State Management:
    - Local state: UI filters, selected date range, expanded views
    - Remote state: Assignment data (via React Query)
    - Context: User authentication, platform connections

Performance Considerations:
    - Uses React Query for automatic caching and background refetching
    - Implements virtualization for lists exceeding 50 assignments
    - Debounces filter changes to reduce API calls
    - Lazy loads assignment detail modals

Accessibility:
    - Full keyboard navigation support
    - ARIA labels for screen readers
    - Focus management for modals and filters
    - Color contrast ratios meet WCAG AA standards

Example Usage:
    ```tsx
    import TeacherDashboard from '@/components/dashboard/TeacherDashboard';

    function App() {
        return (
            <AuthProvider>
                <TeacherDashboard userId="teacher_123" />
            </AuthProvider>
        );
    }
    ```

Author: Kyle Sherman
Created: February 2026
Version: 1.0.0
"""
```

### 3.2 Input Specification

#### 3.2.1 Component Props (External Inputs)

```typescript
interface TeacherDashboardProps {
    /**
     * Authenticated user identifier
     * Used to fetch user-specific assignments and platform connections
     * @required
     * @type {string}
     * @example "teacher_12345"
     */
    userId: string;

    /**
     * Initial date range for assignment display
     * If not provided, defaults to next 30 days
     * @optional
     * @type {DateRange}
     */
    initialDateRange?: DateRange;

    /**
     * Initial platform filter selection
     * If not provided, shows assignments from all platforms
     * @optional
     * @type {string[]}
     * @example ["google_classroom", "powerschool"]
     */
    initialPlatforms?: string[];

    /**
     * Initial course filter selection
     * If not provided, shows assignments from all courses
     * @optional
     * @type {string[]}
     * @example ["course_101", "course_202"]
     */
    initialCourses?: string[];

    /**
     * Callback function when user navigates to platform
     * Used for analytics tracking
     * @optional
     * @type {function}
     */
    onPlatformNavigate?: (platform: string, assignmentId: string) => void;

    /**
     * Callback function when user exports calendar
     * Used for analytics tracking
     * @optional
     * @type {function}
     */
    onExport?: (format: 'ical' | 'google') => void;

    /**
     * Feature flag for showing experimental features
     * @optional
     * @type {boolean}
     * @default false
     */
    enableExperimentalFeatures?: boolean;
}
```

#### 3.2.2 Supporting Type Definitions

```typescript
/**
 * Date range for filtering assignments
 */
interface DateRange {
    /**
     * Start date (inclusive)
     * @type {Date}
     */
    startDate: Date;

    /**
     * End date (inclusive)
     * @type {Date}
     */
    endDate: Date;
}

/**
 * Assignment object from API
 */
interface Assignment {
    /**
     * Unique identifier across all platforms
     * @type {string}
     * @example "gc_123456"
     */
    id: string;

    /**
     * Platform identifier
     * @type {PlatformId}
     */
    platformId: PlatformId;

    /**
     * Native assignment ID from source platform
     * @type {string}
     */
    nativeId: string;

    /**
     * Assignment title
     * @type {string}
     * @example "Chapter 5 Homework: Loops and Conditionals"
     */
    title: string;

    /**
     * Assignment description (may contain HTML)
     * @type {string | null}
     */
    description: string | null;

    /**
     * Due date and time
     * @type {Date | null}
     */
    dueDate: Date | null;

    /**
     * Maximum possible points
     * @type {number | null}
     */
    pointsPossible: number | null;

    /**
     * Associated course identifier
     * @type {string}
     */
    courseId: string;

    /**
     * Course name for display
     * @type {string}
     * @example "AP Computer Science Principles"
     */
    courseName: string;

    /**
     * Assignment type/category
     * @type {AssignmentType}
     */
    assignmentType: AssignmentType;

    /**
     * URL to assignment in native platform
     * @type {string}
     * @example "https://classroom.google.com/c/12345/a/67890"
     */
    url: string;

    /**
     * Number of students who submitted
     * @type {number | null}
     */
    submissionCount?: number | null;

    /**
     * Total number of students in course
     * @type {number | null}
     */
    totalStudents?: number | null;

    /**
     * Platform-specific metadata
     * @type {Record<string, any>}
     */
    metadata: Record<string, any>;
}

/**
 * Platform identifier enum
 */
type PlatformId =
    | 'google_classroom'
    | 'powerschool'
    | 'codehs'
    | 'canvas'
    | 'schoology';

/**
 * Assignment type enum
 */
type AssignmentType =
    | 'homework'
    | 'quiz'
    | 'test'
    | 'project'
    | 'exercise'
    | 'discussion'
    | 'other';

/**
 * Filter state for assignments
 */
interface FilterState {
    /**
     * Date range filter
     * @type {DateRange}
     */
    dateRange: DateRange;

    /**
     * Selected platforms (empty array means all)
     * @type {PlatformId[]}
     */
    platforms: PlatformId[];

    /**
     * Selected courses (empty array means all)
     * @type {string[]}
     */
    courses: string[];

    /**
     * Selected assignment types (empty array means all)
     * @type {AssignmentType[]}
     */
    assignmentTypes: AssignmentType[];

    /**
     * Search query for title/description
     * @type {string}
     */
    searchQuery: string;
}

/**
 * Platform connection status
 */
interface PlatformConnection {
    /**
     * Platform identifier
     * @type {PlatformId}
     */
    platformId: PlatformId;

    /**
     * Connection status
     * @type {ConnectionStatus}
     */
    status: ConnectionStatus;

    /**
     * Last successful sync timestamp
     * @type {Date | null}
     */
    lastSync: Date | null;

    /**
     * Error message if connection failed
     * @type {string | null}
     */
    errorMessage: string | null;
}

/**
 * Connection status enum
 */
type ConnectionStatus =
    | 'connected'
    | 'disconnected'
    | 'error'
    | 'syncing';
```

#### 3.2.3 API Response Structure

```typescript
/**
 * API response for GET /api/assignments
 */
interface AssignmentsResponse {
    /**
     * List of assignments matching filter criteria
     * @type {Assignment[]}
     */
    assignments: Assignment[];

    /**
     * Pagination metadata
     * @type {PaginationMeta}
     */
    meta: PaginationMeta;

    /**
     * Platform connection status
     * @type {PlatformConnection[]}
     */
    connections: PlatformConnection[];
}

/**
 * Pagination metadata
 */
interface PaginationMeta {
    /**
     * Total number of assignments matching filters
     * @type {number}
     */
    total: number;

    /**
     * Current page number (1-indexed)
     * @type {number}
     */
    page: number;

    /**
     * Number of items per page
     * @type {number}
     */
    perPage: number;

    /**
     * Total number of pages
     * @type {number}
     */
    totalPages: number;
}
```

### 3.3 Output Specification

#### 3.3.1 Visual Outputs (Rendered UI)

```typescript
/**
 * Component renders the following visual elements:
 *
 * 1. Header Section
 *    - Dashboard title: "Assignment Dashboard"
 *    - Sync status indicator (icon + timestamp)
 *    - Manual refresh button
 *    - Export button with dropdown (iCal, Google Calendar)
 *
 * 2. Filter Panel
 *    - Date range picker (start date, end date, presets)
 *    - Platform multi-select checkboxes
 *    - Course multi-select dropdown
 *    - Assignment type multi-select dropdown
 *    - Search input field
 *    - "Clear All Filters" button
 *
 * 3. Assignment Timeline
 *    - Grouped by date (Today, Tomorrow, This Week, Later)
 *    - Assignment cards containing:
 *      * Platform badge (colored, with icon)
 *      * Assignment title
 *      * Course name
 *      * Due date/time (formatted, with proximity indicator)
 *      * Points possible
 *      * Submission count (X/Y submitted)
 *      * Assignment type tag
 *      * Quick action: "Open in [Platform]" button
 *    - Empty state when no assignments match filters
 *
 * 4. Loading States
 *    - Skeleton loaders for assignment cards
 *    - Spinner on sync status indicator
 *
 * 5. Error States
 *    - Connection error banner (per platform)
 *    - General error message with retry button
 *
 * 6. Footer Section
 *    - Summary statistics (X assignments shown, Y total)
 *    - Last updated timestamp
 */
```

#### 3.3.2 Event Outputs (User Interactions)

```typescript
/**
 * Events emitted by component
 */
interface DashboardEvents {
    /**
     * User clicked on assignment card
     * Opens assignment in native platform in new tab
     * @emits {AssignmentClickEvent}
     */
    onAssignmentClick: (event: AssignmentClickEvent) => void;

    /**
     * User changed filter selections
     * Triggers new API call with updated filters
     * @emits {FilterChangeEvent}
     */
    onFilterChange: (event: FilterChangeEvent) => void;

    /**
     * User clicked manual refresh button
     * Triggers immediate sync for all connected platforms
     * @emits {RefreshEvent}
     */
    onRefresh: (event: RefreshEvent) => void;

    /**
     * User clicked export button
     * Downloads calendar file or redirects to Google Calendar
     * @emits {ExportEvent}
     */
    onExport: (event: ExportEvent) => void;

    /**
     * User clicked on platform connection error
     * Navigates to settings page to fix connection
     * @emits {NavigateToSettingsEvent}
     */
    onNavigateToSettings: (event: NavigateToSettingsEvent) => void;
}

interface AssignmentClickEvent {
    assignmentId: string;
    platformId: PlatformId;
    url: string;
    timestamp: Date;
}

interface FilterChangeEvent {
    previousFilters: FilterState;
    newFilters: FilterState;
    changedField: keyof FilterState;
    timestamp: Date;
}

interface RefreshEvent {
    platforms: PlatformId[];
    timestamp: Date;
}

interface ExportEvent {
    format: 'ical' | 'google';
    filters: FilterState;
    assignmentCount: number;
    timestamp: Date;
}

interface NavigateToSettingsEvent {
    platformId: PlatformId;
    timestamp: Date;
}
```

#### 3.3.3 State Outputs (Component State)

```typescript
/**
 * Internal component state managed by React hooks
 */
interface DashboardState {
    /**
     * Current filter selections
     * @type {FilterState}
     */
    filters: FilterState;

    /**
     * Assignment data loading status
     * @type {boolean}
     */
    isLoading: boolean;

    /**
     * Error state from API calls
     * @type {Error | null}
     */
    error: Error | null;

    /**
     * Currently expanded assignment card (for mobile)
     * @type {string | null}
     */
    expandedAssignmentId: string | null;

    /**
     * Filter panel visibility (for mobile)
     * @type {boolean}
     */
    isFilterPanelOpen: boolean;

    /**
     * Export dropdown visibility
     * @type {boolean}
     */
    isExportMenuOpen: boolean;
}
```

---

## 4. Logic Model

### 4.1 Component Lifecycle

```
┌─────────────────────────────────────────────────────────────────┐
│                     COMPONENT MOUNT                              │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  1. Initialize State                                             │
│     - Set default filters (next 30 days, all platforms)         │
│     - Apply props overrides (initialDateRange, etc.)            │
│     - Initialize React Query cache                              │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  2. Fetch Platform Connections                                   │
│     API: GET /api/connections                                    │
│     - Retrieve list of connected platforms                       │
│     - Check connection status for each                           │
│     - Store in React Query cache                                 │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  3. Fetch Assignments (Initial Load)                             │
│     API: GET /api/assignments?...filters                         │
│     - Send filter parameters in query string                     │
│     - Receive paginated assignment list                          │
│     - Store in React Query cache                                 │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  4. Render UI                                                    │
│     - Display filter panel                                       │
│     - Render assignment timeline                                 │
│     - Show sync status                                           │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  5. Background Refetch                                           │
│     - React Query automatically refetches every 5 minutes        │
│     - Updates cache without blocking UI                          │
└─────────────────────────────────────────────────────────────────┘
```

### 4.2 Data Transformation Pipeline

```
┌─────────────────────────────────────────────────────────────────┐
│  INPUT: Raw API Response                                         │
│  {                                                               │
│    assignments: [...],                                           │
│    meta: {...},                                                  │
│    connections: [...]                                            │
│  }                                                               │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  TRANSFORM 1: Parse Dates                                        │
│  - Convert ISO 8601 strings to Date objects                      │
│  - Handle null due dates                                         │
│  - Parse lastSync timestamps                                     │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  TRANSFORM 2: Enrich Assignments                                 │
│  - Calculate due date proximity (hours until due)                │
│  - Determine urgency level (critical, warning, normal)           │
│  - Calculate submission percentage                               │
│  - Add platform display metadata (name, color, icon)             │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  TRANSFORM 3: Group by Date                                      │
│  - TODAY: Due within 24 hours                                    │
│  - TOMORROW: Due within 48 hours                                 │
│  - THIS_WEEK: Due within 7 days                                  │
│  - LATER: Due after 7 days                                       │
│  - NO_DUE_DATE: Assignments without due dates                    │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  TRANSFORM 4: Sort Within Groups                                 │
│  - Primary sort: Due date (ascending)                            │
│  - Secondary sort: Platform (alphabetical)                       │
│  - Tertiary sort: Course name (alphabetical)                     │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  OUTPUT: Grouped & Sorted Assignments                            │
│  {                                                               │
│    today: [Assignment, Assignment, ...],                         │
│    tomorrow: [...],                                              │
│    thisWeek: [...],                                              │
│    later: [...],                                                 │
│    noDueDate: [...]                                              │
│  }                                                               │
└─────────────────────────────────────────────────────────────────┘
```

### 4.3 Filter Logic Model

```
┌─────────────────────────────────────────────────────────────────┐
│  USER ACTION: Filter Change                                      │
│  (e.g., User selects date range "Next 7 Days")                  │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 1: Update Local State                                      │
│  - Merge new filter value with existing FilterState             │
│  - Trigger re-render with loading indicator                      │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 2: Debounce (300ms)                                        │
│  - Wait for user to finish making changes                        │
│  - Cancel previous pending requests                              │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 3: Build Query Parameters                                  │
│  filters: {                                                      │
│    dateRange: { start: '2026-02-20', end: '2026-02-27' },       │
│    platforms: ['google_classroom'],                             │
│    courses: [],                                                  │
│    types: []                                                     │
│  }                                                               │
│  ↓                                                               │
│  query string: ?start=2026-02-20&end=2026-02-27&                │
│                platform=google_classroom                         │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 4: API Request                                             │
│  GET /api/assignments?start=2026-02-20&end=2026-02-27&...       │
│  - React Query checks cache first                               │
│  - If cache miss, sends HTTP request                             │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 5: Update UI                                               │
│  - Remove loading spinner                                        │
│  - Render new assignment list                                    │
│  - Update summary statistics                                     │
│  - Persist filter state to localStorage                          │
└─────────────────────────────────────────────────────────────────┘
```

### 4.4 Assignment Click Logic

```
┌─────────────────────────────────────────────────────────────────┐
│  USER ACTION: Click Assignment Card                              │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 1: Extract Assignment Data                                 │
│  - Get assignment.url (native platform URL)                      │
│  - Get assignment.platformId                                     │
│  - Get assignment.id                                             │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 2: Analytics Tracking (Optional)                           │
│  - Call onPlatformNavigate callback if provided                  │
│  - Log event to analytics service                                │
│  - Track: userId, platformId, assignmentId, timestamp            │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 3: Open Native Platform                                    │
│  - window.open(assignment.url, '_blank')                         │
│  - New tab opens to assignment in Google Classroom/PS/CodeHS    │
│  - EduSync tab remains open (no navigation)                      │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 4: Update UI State (Optional)                              │
│  - Mark assignment as "viewed" (visual indicator)                │
│  - Update lastViewed timestamp in cache                          │
└─────────────────────────────────────────────────────────────────┘
```

### 4.5 Sync Status Logic

```
┌─────────────────────────────────────────────────────────────────┐
│  BACKGROUND PROCESS: Platform Sync Status Check                  │
│  (Runs every 30 seconds)                                         │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 1: Fetch Connection Status                                 │
│  GET /api/connections                                            │
│  Response: [                                                     │
│    { platformId: 'google_classroom', status: 'connected',       │
│      lastSync: '2026-02-20T10:30:00Z' },                        │
│    { platformId: 'powerschool', status: 'error',                │
│      errorMessage: 'Invalid credentials' }                       │
│  ]                                                               │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 2: Calculate Freshness                                     │
│  For each platform:                                              │
│  - hoursAgo = now - lastSync                                     │
│  - If hoursAgo < 1: "Just now" (green)                          │
│  - If hoursAgo < 12: "X hours ago" (green)                      │
│  - If hoursAgo < 24: "X hours ago" (yellow)                     │
│  - If hoursAgo >= 24: "X days ago" (red)                        │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 3: Display Status                                          │
│  - Show sync icon (rotating if syncing)                          │
│  - Show timestamp of oldest successful sync                      │
│  - If any platform has error: show warning banner               │
│  - Banner links to Settings to fix connection                    │
└─────────────────────────────────────────────────────────────────┘
```

### 4.6 Error Handling Logic

```
┌─────────────────────────────────────────────────────────────────┐
│  ERROR SCENARIO: API Request Fails                               │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 1: Categorize Error                                        │
│  - Network Error (offline, timeout)                              │
│  - Authentication Error (401, 403)                               │
│  - Server Error (500, 502, 503)                                  │
│  - Validation Error (400)                                        │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 2: Display User-Friendly Message                           │
│  - Network Error: "Can't connect. Check your internet."         │
│  - Auth Error: "Session expired. Please log in again."          │
│  - Server Error: "EduSync is having issues. Try again soon."    │
│  - Validation Error: Show specific field errors                  │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 3: Provide Recovery Options                                │
│  - "Retry" button (attempts same request)                        │
│  - "Use Cached Data" (shows stale data with warning)            │
│  - "Contact Support" link                                        │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 4: Log Error for Debugging                                 │
│  - Send to error tracking service (Sentry)                       │
│  - Include: userId, filters, timestamp, stack trace              │
└─────────────────────────────────────────────────────────────────┘
```

---

## 5. Source Code Implementation

### 5.1 Component Structure

```
src/components/dashboard/
├── TeacherDashboard.tsx          # Main component (below)
├── AssignmentTimeline.tsx        # Timeline rendering
├── AssignmentCard.tsx            # Individual assignment card
├── FilterPanel.tsx               # Filter controls
├── DateRangePicker.tsx           # Date selection
├── PlatformFilter.tsx            # Platform checkboxes
├── SyncStatusIndicator.tsx       # Sync status display
└── ExportMenu.tsx                # Export dropdown
```

### 5.2 Main Component Implementation

```typescript
// src/components/dashboard/TeacherDashboard.tsx

import React, { useState, useEffect, useCallback, useMemo } from 'react';
import { useQuery, useMutation, useQueryClient } from '@tanstack/react-query';
import { addDays, startOfDay, endOfDay, format, differenceInHours } from 'date-fns';
import {
    RefreshCw,
    Download,
    AlertCircle,
    CheckCircle,
    Clock
} from 'lucide-react';

import AssignmentTimeline from './AssignmentTimeline';
import FilterPanel from './FilterPanel';
import SyncStatusIndicator from './SyncStatusIndicator';
import ExportMenu from './ExportMenu';

import {
    fetchAssignments,
    fetchPlatformConnections,
    triggerManualSync
} from '@/services/api';

import type {
    TeacherDashboardProps,
    FilterState,
    Assignment,
    PlatformConnection,
    DateRange,
    PlatformId,
    AssignmentType
} from '@/types';

/**
 * Teacher Dashboard Component
 *
 * Primary interface for viewing aggregated assignments across platforms.
 * See comprehensive docstring at top of file for full documentation.
 */
const TeacherDashboard: React.FC<TeacherDashboardProps> = ({
    userId,
    initialDateRange,
    initialPlatforms = [],
    initialCourses = [],
    onPlatformNavigate,
    onExport,
    enableExperimentalFeatures = false
}) => {
    // ====================================================================
    // STATE MANAGEMENT
    // ====================================================================

    const queryClient = useQueryClient();

    // Initialize default date range (next 30 days)
    const defaultDateRange: DateRange = useMemo(() => ({
        startDate: startOfDay(new Date()),
        endDate: endOfDay(addDays(new Date(), 30))
    }), []);

    // Filter state
    const [filters, setFilters] = useState<FilterState>({
        dateRange: initialDateRange || defaultDateRange,
        platforms: initialPlatforms,
        courses: initialCourses,
        assignmentTypes: [],
        searchQuery: ''
    });

    // UI state
    const [isFilterPanelOpen, setIsFilterPanelOpen] = useState(true);
    const [isExportMenuOpen, setIsExportMenuOpen] = useState(false);
    const [expandedAssignmentId, setExpandedAssignmentId] = useState<string | null>(null);

    // ====================================================================
    // DATA FETCHING
    // ====================================================================

    /**
     * Fetch platform connections to check sync status
     * Refetches every 30 seconds to keep status current
     */
    const {
        data: connectionsData,
        isLoading: connectionsLoading,
        error: connectionsError
    } = useQuery({
        queryKey: ['connections', userId],
        queryFn: () => fetchPlatformConnections(userId),
        refetchInterval: 30000, // 30 seconds
        staleTime: 30000
    });

    /**
     * Fetch assignments based on current filters
     * Uses React Query for automatic caching and background refetching
     */
    const {
        data: assignmentsData,
        isLoading: assignmentsLoading,
        error: assignmentsError,
        refetch: refetchAssignments
    } = useQuery({
        queryKey: ['assignments', userId, filters],
        queryFn: () => fetchAssignments(userId, filters),
        refetchInterval: 300000, // 5 minutes
        staleTime: 60000, // 1 minute
        keepPreviousData: true // Prevent UI flicker during refetch
    });

    /**
     * Manual sync mutation
     * Triggers immediate sync for all connected platforms
     */
    const syncMutation = useMutation({
        mutationFn: () => triggerManualSync(userId),
        onSuccess: () => {
            // Invalidate queries to trigger refetch
            queryClient.invalidateQueries(['assignments', userId]);
            queryClient.invalidateQueries(['connections', userId]);
        }
    });

    // ====================================================================
    // DATA TRANSFORMATION
    // ====================================================================

    /**
     * Transform and group assignments by due date proximity
     * Applies enrichment logic: urgency levels, submission percentages, etc.
     */
    const groupedAssignments = useMemo(() => {
        if (!assignmentsData?.assignments) {
            return {
                today: [],
                tomorrow: [],
                thisWeek: [],
                later: [],
                noDueDate: []
            };
        }

        const now = new Date();
        const groups = {
            today: [] as Assignment[],
            tomorrow: [] as Assignment[],
            thisWeek: [] as Assignment[],
            later: [] as Assignment[],
            noDueDate: [] as Assignment[]
        };

        assignmentsData.assignments.forEach(assignment => {
            // Parse due date
            const dueDate = assignment.dueDate
                ? new Date(assignment.dueDate)
                : null;

            if (!dueDate) {
                groups.noDueDate.push(assignment);
                return;
            }

            const hoursUntilDue = differenceInHours(dueDate, now);

            // Categorize by proximity
            if (hoursUntilDue <= 24) {
                groups.today.push(assignment);
            } else if (hoursUntilDue <= 48) {
                groups.tomorrow.push(assignment);
            } else if (hoursUntilDue <= 168) { // 7 days
                groups.thisWeek.push(assignment);
            } else {
                groups.later.push(assignment);
            }
        });

        // Sort within each group
        const sortAssignments = (a: Assignment, b: Assignment) => {
            // Primary: due date
            if (a.dueDate && b.dueDate) {
                const dateCompare = new Date(a.dueDate).getTime() - new Date(b.dueDate).getTime();
                if (dateCompare !== 0) return dateCompare;
            }

            // Secondary: platform
            const platformCompare = a.platformId.localeCompare(b.platformId);
            if (platformCompare !== 0) return platformCompare;

            // Tertiary: course name
            return a.courseName.localeCompare(b.courseName);
        };

        Object.keys(groups).forEach(key => {
            groups[key as keyof typeof groups].sort(sortAssignments);
        });

        return groups;
    }, [assignmentsData]);

    /**
     * Calculate summary statistics
     */
    const statistics = useMemo(() => {
        const total = assignmentsData?.meta?.total || 0;
        const shown = assignmentsData?.assignments?.length || 0;

        return {
            total,
            shown,
            filtered: total > shown
        };
    }, [assignmentsData]);

    /**
     * Determine overall sync status
     * Shows oldest sync time and any connection errors
     */
    const syncStatus = useMemo(() => {
        if (!connectionsData?.connections) {
            return {
                status: 'unknown' as const,
                message: 'Loading...',
                lastSync: null
            };
        }

        const connections = connectionsData.connections;

        // Check for errors
        const hasErrors = connections.some(c => c.status === 'error');
        if (hasErrors) {
            return {
                status: 'error' as const,
                message: 'Some platforms disconnected',
                lastSync: null
            };
        }

        // Check for syncing
        const isSyncing = connections.some(c => c.status === 'syncing');
        if (isSyncing) {
            return {
                status: 'syncing' as const,
                message: 'Syncing...',
                lastSync: null
            };
        }

        // Find oldest sync
        const syncs = connections
            .filter(c => c.lastSync)
            .map(c => new Date(c.lastSync!).getTime());

        if (syncs.length === 0) {
            return {
                status: 'unknown' as const,
                message: 'Never synced',
                lastSync: null
            };
        }

        const oldestSync = new Date(Math.min(...syncs));
        const hoursAgo = differenceInHours(new Date(), oldestSync);

        let status: 'fresh' | 'stale' | 'very_stale';
        if (hoursAgo < 12) {
            status = 'fresh';
        } else if (hoursAgo < 24) {
            status = 'stale';
        } else {
            status = 'very_stale';
        }

        return {
            status,
            message: formatSyncTime(oldestSync),
            lastSync: oldestSync
        };
    }, [connectionsData]);

    // ====================================================================
    // EVENT HANDLERS
    // ====================================================================

    /**
     * Handle filter changes
     * Debounced to prevent excessive API calls
     */
    const handleFilterChange = useCallback((newFilters: Partial<FilterState>) => {
        setFilters(prev => ({
            ...prev,
            ...newFilters
        }));

        // Persist to localStorage for session continuity
        localStorage.setItem('dashboardFilters', JSON.stringify({
            ...filters,
            ...newFilters
        }));
    }, [filters]);

    /**
     * Handle assignment card click
     * Opens assignment in native platform
     */
    const handleAssignmentClick = useCallback((assignment: Assignment) => {
        // Analytics tracking
        if (onPlatformNavigate) {
            onPlatformNavigate(assignment.platformId, assignment.id);
        }

        // Open in new tab
        window.open(assignment.url, '_blank', 'noopener,noreferrer');

        // Optional: Mark as viewed
        // updateAssignmentViewed(assignment.id);
    }, [onPlatformNavigate]);

    /**
     * Handle manual refresh
     * Triggers sync for all platforms
     */
    const handleRefresh = useCallback(() => {
        syncMutation.mutate();
    }, [syncMutation]);

    /**
     * Handle export
     * Downloads calendar file or redirects to Google Calendar
     */
    const handleExport = useCallback((format: 'ical' | 'google') => {
        if (onExport) {
            onExport(format);
        }

        // Implementation would generate calendar file
        // For now, placeholder
        console.log(`Exporting ${statistics.shown} assignments as ${format}`);
    }, [onExport, statistics.shown]);

    /**
     * Clear all filters
     * Resets to default state
     */
    const handleClearFilters = useCallback(() => {
        setFilters({
            dateRange: defaultDateRange,
            platforms: [],
            courses: [],
            assignmentTypes: [],
            searchQuery: ''
        });

        localStorage.removeItem('dashboardFilters');
    }, [defaultDateRange]);

    // ====================================================================
    // EFFECTS
    // ====================================================================

    /**
     * Load saved filters from localStorage on mount
     */
    useEffect(() => {
        const saved = localStorage.getItem('dashboardFilters');
        if (saved) {
            try {
                const parsed = JSON.parse(saved);
                // Convert string dates back to Date objects
                parsed.dateRange.startDate = new Date(parsed.dateRange.startDate);
                parsed.dateRange.endDate = new Date(parsed.dateRange.endDate);
                setFilters(parsed);
            } catch (e) {
                console.error('Failed to parse saved filters:', e);
            }
        }
    }, []);

    // ====================================================================
    // RENDER
    // ====================================================================

    return (
        <div className="min-h-screen bg-gray-50">
            {/* Header */}
            <header className="bg-white border-b border-gray-200 sticky top-0 z-10">
                <div className="max-w-7xl mx-auto px-4 sm:px-6 lg:px-8 py-4">
                    <div className="flex items-center justify-between">
                        {/* Title */}
                        <div>
                            <h1 className="text-2xl font-bold text-gray-900">
                                Assignment Dashboard
                            </h1>
                            <p className="text-sm text-gray-500 mt-1">
                                {statistics.shown} of {statistics.total} assignments
                            </p>
                        </div>

                        {/* Actions */}
                        <div className="flex items-center gap-3">
                            {/* Sync Status */}
                            <SyncStatusIndicator
                                status={syncStatus.status}
                                message={syncStatus.message}
                                lastSync={syncStatus.lastSync}
                            />

                            {/* Refresh Button */}
                            <button
                                onClick={handleRefresh}
                                disabled={syncMutation.isLoading}
                                className="inline-flex items-center px-3 py-2 border border-gray-300 rounded-md text-sm font-medium text-gray-700 bg-white hover:bg-gray-50 disabled:opacity-50"
                            >
                                <RefreshCw
                                    className={`h-4 w-4 mr-2 ${syncMutation.isLoading ? 'animate-spin' : ''}`}
                                />
                                Refresh
                            </button>

                            {/* Export Button */}
                            <ExportMenu
                                isOpen={isExportMenuOpen}
                                onToggle={() => setIsExportMenuOpen(!isExportMenuOpen)}
                                onExport={handleExport}
                            />
                        </div>
                    </div>
                </div>
            </header>

            {/* Connection Error Banner */}
            {syncStatus.status === 'error' && (
                <div className="bg-red-50 border-b border-red-200">
                    <div className="max-w-7xl mx-auto px-4 sm:px-6 lg:px-8 py-3">
                        <div className="flex items-center">
                            <AlertCircle className="h-5 w-5 text-red-600 mr-3" />
                            <p className="text-sm text-red-800">
                                Some platforms are disconnected.
                                <button className="font-medium underline ml-1">
                                    Fix in Settings
                                </button>
                            </p>
                        </div>
                    </div>
                </div>
            )}

            {/* Main Content */}
            <main className="max-w-7xl mx-auto px-4 sm:px-6 lg:px-8 py-6">
                <div className="grid grid-cols-12 gap-6">
                    {/* Filter Panel */}
                    <aside
                        className={`col-span-12 lg:col-span-3 ${isFilterPanelOpen ? '' : 'hidden lg:block'}`}
                    >
                        <FilterPanel
                            filters={filters}
                            onFilterChange={handleFilterChange}
                            onClearAll={handleClearFilters}
                            platforms={connectionsData?.connections || []}
                        />
                    </aside>

                    {/* Assignment Timeline */}
                    <section className="col-span-12 lg:col-span-9">
                        {assignmentsLoading ? (
                            <LoadingSkeleton />
                        ) : assignmentsError ? (
                            <ErrorState error={assignmentsError} onRetry={refetchAssignments} />
                        ) : (
                            <AssignmentTimeline
                                groups={groupedAssignments}
                                onAssignmentClick={handleAssignmentClick}
                                expandedId={expandedAssignmentId}
                                onExpandToggle={setExpandedAssignmentId}
                            />
                        )}
                    </section>
                </div>
            </main>

            {/* Footer */}
            <footer className="bg-white border-t border-gray-200 mt-12">
                <div className="max-w-7xl mx-auto px-4 sm:px-6 lg:px-8 py-4">
                    <div className="flex items-center justify-between text-sm text-gray-500">
                        <div>
                            Last updated: {syncStatus.lastSync
                                ? format(syncStatus.lastSync, 'MMM d, yyyy h:mm a')
                                : 'Never'
                            }
                        </div>
                        <div>
                            EduSync v1.0.0
                        </div>
                    </div>
                </div>
            </footer>
        </div>
    );
};

// ====================================================================
// HELPER COMPONENTS
// ====================================================================

/**
 * Loading skeleton for assignment cards
 */
const LoadingSkeleton: React.FC = () => {
    return (
        <div className="space-y-4">
            {[1, 2, 3, 4, 5].map(i => (
                <div key={i} className="bg-white rounded-lg p-4 animate-pulse">
                    <div className="h-4 bg-gray-200 rounded w-3/4 mb-3"></div>
                    <div className="h-3 bg-gray-200 rounded w-1/2 mb-2"></div>
                    <div className="h-3 bg-gray-200 rounded w-1/4"></div>
                </div>
            ))}
        </div>
    );
};

/**
 * Error state with retry option
 */
const ErrorState: React.FC<{ error: any; onRetry: () => void }> = ({ error, onRetry }) => {
    return (
        <div className="bg-white rounded-lg p-12 text-center">
            <AlertCircle className="h-12 w-12 text-red-500 mx-auto mb-4" />
            <h3 className="text-lg font-medium text-gray-900 mb-2">
                Failed to load assignments
            </h3>
            <p className="text-sm text-gray-500 mb-6">
                {error.message || 'An unexpected error occurred'}
            </p>
            <button
                onClick={onRetry}
                className="inline-flex items-center px-4 py-2 border border-transparent rounded-md shadow-sm text-sm font-medium text-white bg-blue-600 hover:bg-blue-700"
            >
                Try Again
            </button>
        </div>
    );
};

// ====================================================================
// UTILITY FUNCTIONS
// ====================================================================

/**
 * Format sync timestamp for display
 */
function formatSyncTime(date: Date): string {
    const hoursAgo = differenceInHours(new Date(), date);

    if (hoursAgo < 1) {
        return 'Just now';
    } else if (hoursAgo < 24) {
        return `${hoursAgo} hour${hoursAgo === 1 ? '' : 's'} ago`;
    } else {
        const daysAgo = Math.floor(hoursAgo / 24);
        return `${daysAgo} day${daysAgo === 1 ? '' : 's'} ago`;
    }
}

export default TeacherDashboard;
```

### 5.3 Supporting Type Definitions File

```typescript
// src/types/index.ts

export interface TeacherDashboardProps {
    userId: string;
    initialDateRange?: DateRange;
    initialPlatforms?: string[];
    initialCourses?: string[];
    onPlatformNavigate?: (platform: string, assignmentId: string) => void;
    onExport?: (format: 'ical' | 'google') => void;
    enableExperimentalFeatures?: boolean;
}

export interface DateRange {
    startDate: Date;
    endDate: Date;
}

export interface Assignment {
    id: string;
    platformId: PlatformId;
    nativeId: string;
    title: string;
    description: string | null;
    dueDate: Date | null;
    pointsPossible: number | null;
    courseId: string;
    courseName: string;
    assignmentType: AssignmentType;
    url: string;
    submissionCount?: number | null;
    totalStudents?: number | null;
    metadata: Record<string, any>;
}

export type PlatformId =
    | 'google_classroom'
    | 'powerschool'
    | 'codehs'
    | 'canvas'
    | 'schoology';

export type AssignmentType =
    | 'homework'
    | 'quiz'
    | 'test'
    | 'project'
    | 'exercise'
    | 'discussion'
    | 'other';

export interface FilterState {
    dateRange: DateRange;
    platforms: PlatformId[];
    courses: string[];
    assignmentTypes: AssignmentType[];
    searchQuery: string;
}

export interface PlatformConnection {
    platformId: PlatformId;
    status: ConnectionStatus;
    lastSync: Date | null;
    errorMessage: string | null;
}

export type ConnectionStatus =
    | 'connected'
    | 'disconnected'
    | 'error'
    | 'syncing';

export interface AssignmentsResponse {
    assignments: Assignment[];
    meta: PaginationMeta;
    connections: PlatformConnection[];
}

export interface PaginationMeta {
    total: number;
    page: number;
    perPage: number;
    totalPages: number;
}
```

---

## 6. Testing Strategy

### 6.1 Unit Tests

```typescript
// __tests__/TeacherDashboard.test.tsx

import { render, screen, waitFor } from '@testing-library/react';
import userEvent from '@testing-library/user-event';
import { QueryClient, QueryClientProvider } from '@tanstack/react-query';
import TeacherDashboard from '../TeacherDashboard';

describe('TeacherDashboard', () => {
    let queryClient: QueryClient;

    beforeEach(() => {
        queryClient = new QueryClient({
            defaultOptions: {
                queries: { retry: false }
            }
        });
    });

    test('renders dashboard title', () => {
        render(
            <QueryClientProvider client={queryClient}>
                <TeacherDashboard userId="teacher_123" />
            </QueryClientProvider>
        );

        expect(screen.getByText('Assignment Dashboard')).toBeInTheDocument();
    });

    test('displays loading state initially', () => {
        render(
            <QueryClientProvider client={queryClient}>
                <TeacherDashboard userId="teacher_123" />
            </QueryClientProvider>
        );

        expect(screen.getByTestId('loading-skeleton')).toBeInTheDocument();
    });

    test('groups assignments by due date correctly', async () => {
        // Mock API response
        const mockAssignments = [
            { id: '1', title: 'Due Today', dueDate: new Date() },
            { id: '2', title: 'Due Tomorrow', dueDate: addDays(new Date(), 1) },
            { id: '3', title: 'Due Next Week', dueDate: addDays(new Date(), 5) }
        ];

        // Test grouping logic
        // ...
    });

    test('filters assignments when filter changed', async () => {
        render(
            <QueryClientProvider client={queryClient}>
                <TeacherDashboard userId="teacher_123" />
            </QueryClientProvider>
        );

        const platformFilter = screen.getByLabelText('Google Classroom');
        await userEvent.click(platformFilter);

        await waitFor(() => {
            // Verify filtered results
        });
    });
});
```

### 6.2 Integration Tests

```typescript
// __tests__/integration/TeacherDashboard.integration.test.tsx

import { render, screen, waitFor } from '@testing-library/react';
import { rest } from 'msw';
import { setupServer } from 'msw/node';
import TeacherDashboard from '../TeacherDashboard';

const server = setupServer(
    rest.get('/api/assignments', (req, res, ctx) => {
        return res(ctx.json({
            assignments: mockAssignments,
            meta: { total: 10, page: 1, perPage: 10, totalPages: 1 },
            connections: mockConnections
        }));
    })
);

beforeAll(() => server.listen());
afterEach(() => server.resetHandlers());
afterAll(() => server.close());

test('full user flow: filter and click assignment', async () => {
    render(<TeacherDashboard userId="teacher_123" />);

    // Wait for data to load
    await waitFor(() => {
        expect(screen.getByText('Chapter 5 Homework')).toBeInTheDocument();
    });

    // Click filter
    // ...

    // Click assignment
    // ...

    // Verify navigation
    // ...
});
```

---

**End of Document**